In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType, TimestampType
from dotenv import load_dotenv
import os
import json
import time
import boto3
from pyspark.sql import SparkSession
from datetime import datetime 

In [26]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [27]:
#spark.stop()

In [28]:
from pyspark.sql import SparkSession

# Path to your local JAR files
local_jars = "/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/jars/aws-java-sdk-kinesis-1.12.364.jar"

# Initialize Spark session with Delta and S3 settings
spark = SparkSession.builder \
    .appName("KinesisToDeltaLake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0,org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID) \
    .config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.session.token", AWS_SESSION_TOKEN) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.sql.files.maxPartitionBytes", "134217728") \
    .getOrCreate()

# Optional: Adjust logging level
spark.sparkContext.setLogLevel("WARN")

In [29]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [30]:
# Define the schema
schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("total_weight", DoubleType(), True),
    StructField("total_volume", DoubleType(), True),
    StructField("total_price", DoubleType(), True),
    StructField("order_timestamp", TimestampType(), True),
    StructField("status", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True)
])

In [31]:
# Function to convert the timestamps to proper datetime objects
def convert_timestamps(orders):
    for order in orders:
        if isinstance(order['order_timestamp'], str):
            # Only convert if the timestamp is a string
            order['order_timestamp'] = datetime.strptime(order['order_timestamp'], '%Y-%m-%d %H:%M:%S')
    return orders

# Function to convert the location field from MapType to StructType
def transform_location(order):
    if isinstance(order['location'], dict):
        location_data = order['location']
        order['location'] = {
            "address": location_data['address'],
            "lat": float(location_data['lat']),
            "lon": float(location_data['lon'])
        }
    return order

# Function to get the shard iterator
def get_shard_iterator(stream_name, shard_id):
    response = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType='LATEST'  # or 'LATEST' for new records
    )
    return response['ShardIterator']

# Function to read records from the Kinesis stream
def read_kinesis_records(stream_name, shard_iterator):
    while True:
        response = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=100)
        records = response['Records']
        for record in records:
            # No need to base64 decode, just parse the data directly
            order_data = record['Data']
            order = json.loads(order_data)
            print("Received order:", order)

        # Update the shard iterator for the next batch of records
        shard_iterator = response['NextShardIterator']

        # Sleep to avoid hitting API rate limits
        time.sleep(1)

In [32]:
import time
import json

# Initialize buffer, limits, and thresholds
order_buffer = []
weight_threshold = 15  # Example weight threshold (can be adjusted)
volume_threshold = 500  # Example volume threshold (can be adjusted)
time_threshold = 60 * 30 # Example time threshold (e.g., 30 minutes)
buffer_limit = 1000       # Optional, max number of orders in the buffer
last_flush_time = time.time()

# S3 path for Delta table
delta_table_path = "s3a://orders-for-dispatch/dispatching"

# Dispatcher function to read Kinesis records, accumulate, and process them
def dispatcher(shard_iterator):
    global order_buffer, last_flush_time

    while True:
        # Fetch records from Kinesis using shard_iterator
        response = kinesis_client.get_records(ShardIterator=shard_iterator)
        records = response['Records']

        # Parse and accumulate orders
        for record in records:
            order_data = json.loads(record['Data'])
            order_buffer.append(order_data)
            print(f"Accumulated order: {order_data}")

        # Convert timestamps if necessary
        order_buffer = convert_timestamps(order_buffer)

        # Calculate accumulated weight and volume
        total_weight = sum(order['total_weight'] for order in order_buffer)
        total_volume = sum(order['total_volume'] for order in order_buffer)
        time_elapsed = time.time() - last_flush_time

        # Check if any thresholds are met (weight, volume, or time)
        if total_weight >= weight_threshold or total_volume >= volume_threshold or time_elapsed >= time_threshold:
            print(f"Threshold met: Dispatching {len(order_buffer)} orders.")
            
            # Update the status of all orders to 'DISPATCH_READY'
            for order in order_buffer:
                order['status'] = 'READY_FOR_DISPATCH'

            # Convert buffer to DataFrame and write to Delta table
            df = spark.createDataFrame(order_buffer, schema=schema)
            df.write.format("delta").mode("append").save(delta_table_path)
            print(f"Saved {len(order_buffer)} orders to Delta table.")
            
            # Clear the buffer and reset flush time
            order_buffer.clear()
            last_flush_time = time.time()

        # Update shard iterator for the next batch
        shard_iterator = response['NextShardIterator']
        time.sleep(2)  # Sleep to avoid rate limits

# Get the shard iterator
stream_name = 'OrderStreamForDispatching'  # Replace with your stream name
shard_id = 'shardId-000000000000'  # Get the shard ID from the stream's details
shard_iterator = get_shard_iterator(stream_name, shard_id)

# Start the dispatcher
dispatcher(shard_iterator)


Accumulated order: {'order_id': 'a2cd0a55-1478-4547-95a0-604e9064c322', 'customer_id': 'cus-d352a54f-081c-474f-9385-4cacfeb28550', 'total_weight': 50.24349559200699, 'total_volume': 230.2995934060175, 'total_price': 464.91533542238864, 'order_timestamp': '2024-10-03 03:28:51', 'status': 'RECEIVED', 'lat': 40.4144975398355, 'lon': -3.6476090614891197}
Accumulated order: {'order_id': 'bfc9983d-23f7-4163-8c03-98cc7a041555', 'customer_id': 'cus-f1595d2d-7f14-455e-95f8-3cfa96d040e4', 'total_weight': 48.517479450213614, 'total_volume': 151.80363378642684, 'total_price': 576.5663611345955, 'order_timestamp': '2024-10-03 03:28:52', 'status': 'RECEIVED', 'lat': 40.58429278033965, 'lon': -3.6723275932801482}
Threshold met: Dispatching 2 orders.


Saved 2 orders to Delta table.
Accumulated order: {'order_id': '5745fd6b-50c3-4312-bb01-4166198341fe', 'customer_id': 'cus-2260441b-7585-4cff-b1e9-f9787083a80b', 'total_weight': 58.88165842260217, 'total_volume': 306.4266408049731, 'total_price': 111.27774445952106, 'order_timestamp': '2024-10-03 03:28:53', 'status': 'RECEIVED', 'lat': 40.428046957340584, 'lon': -3.576625841448611}
Accumulated order: {'order_id': '672832ba-7972-4cb3-bd63-55d47800d25c', 'customer_id': 'cus-1e36370f-561f-4079-b666-c2e7723f2c0e', 'total_weight': 44.15439393755439, 'total_volume': 128.28959664119728, 'total_price': 111.22498346602471, 'order_timestamp': '2024-10-03 03:28:58', 'status': 'RECEIVED', 'lat': 40.35225661127856, 'lon': -3.7780006064006884}
Accumulated order: {'order_id': '6f0b12bc-7dfc-4432-8b19-25a1d545ee26', 'customer_id': 'cus-c7255b5b-f67f-4f66-b5dc-1d340ab2cbeb', 'total_weight': 30.93436584680427, 'total_volume': 361.3511529789953, 'total_price': 109.69888627248581, 'order_timestamp': '2024

Saved 3 orders to Delta table.
Accumulated order: {'order_id': '84480db6-91fe-440a-88a1-4db7e1a33d75', 'customer_id': 'cus-30e3b389-4422-45f2-9cd2-88786e803733', 'total_weight': 72.1901243513759, 'total_volume': 73.28362535583715, 'total_price': 677.8554469928214, 'order_timestamp': '2024-10-03 03:29:04', 'status': 'RECEIVED', 'lat': 40.56922872974951, 'lon': -3.7982647031500174}
Accumulated order: {'order_id': 'bd0b02bd-31e0-4880-928a-c98faac904f3', 'customer_id': 'cus-a939ebd7-17ee-4f03-9d66-0143e6df5161', 'total_weight': 88.95039929601127, 'total_volume': 267.78345207028593, 'total_price': 655.5447554905642, 'order_timestamp': '2024-10-03 03:29:05', 'status': 'RECEIVED', 'lat': 40.55046202585554, 'lon': -3.706952056857951}
Accumulated order: {'order_id': 'edcf7ff2-66b5-443f-b6a2-d0ddf01f1b30', 'customer_id': 'cus-2a0cddf4-cfc0-45f8-8502-abe13dee4284', 'total_weight': 17.259033231844175, 'total_volume': 445.6190094796965, 'total_price': 550.4551319757289, 'order_timestamp': '2024-10-

Saved 3 orders to Delta table.
Accumulated order: {'order_id': 'b961bacd-1430-495e-9f16-97dcc45af8f8', 'customer_id': 'cus-0c77cf12-ac06-43b9-8a79-31afc67d7de1', 'total_weight': 3.1571422581544826, 'total_volume': 298.11847062150247, 'total_price': 731.8285689833602, 'order_timestamp': '2024-10-03 03:29:13', 'status': 'RECEIVED', 'lat': 40.64490857977663, 'lon': -3.652433484248254}
Accumulated order: {'order_id': 'a4079fc6-a8d5-4c0c-b527-f80c493d582e', 'customer_id': 'cus-1c2638df-a1c1-4e5e-89a4-ea6191fa09fb', 'total_weight': 59.691198598938726, 'total_volume': 467.8211192765681, 'total_price': 644.943878516838, 'order_timestamp': '2024-10-03 03:29:16', 'status': 'RECEIVED', 'lat': 40.41386973125359, 'lon': -3.6560283921570176}
Threshold met: Dispatching 2 orders.


Saved 2 orders to Delta table.
Accumulated order: {'order_id': '5399273a-82e1-4ca4-b437-effff04d670d', 'customer_id': 'cus-99826dbf-a9fe-431b-95e7-eb7e01c9e2b2', 'total_weight': 96.25855669558426, 'total_volume': 170.38916914958972, 'total_price': 784.2206543526623, 'order_timestamp': '2024-10-03 03:29:19', 'status': 'RECEIVED', 'lat': 40.54888623050556, 'lon': -3.744010416526562}
Accumulated order: {'order_id': 'f492198f-bb13-4c07-ad78-1fe6e6a16849', 'customer_id': 'cus-2f527c49-a8e6-4d82-b936-2d6a4e1a752f', 'total_weight': 30.680417031314136, 'total_volume': 348.9601670537983, 'total_price': 292.0274341356672, 'order_timestamp': '2024-10-03 03:29:21', 'status': 'RECEIVED', 'lat': 40.48968175161479, 'lon': -3.820339619194109}
Accumulated order: {'order_id': '42661805-2649-4d4f-a1cc-ede1cfea2050', 'customer_id': 'cus-313f79db-bfc4-4775-b765-05525bc0f73f', 'total_weight': 82.88947462279884, 'total_volume': 89.02021135773377, 'total_price': 243.52158147628816, 'order_timestamp': '2024-10

...

Saved 1 orders to Delta table.
Accumulated order: {'order_id': 'a13d4038-26cd-4628-8e63-8acbd171a1e3', 'customer_id': 'cus-f212afed-e5aa-4fc1-a20c-82e51772e119', 'total_weight': 5.195906521139581, 'total_volume': 53.17527548648447, 'total_price': 616.9570752913897, 'order_timestamp': '2024-10-03 03:31:33', 'status': 'RECEIVED', 'lat': 40.31713972576694, 'lon': -3.7057502415603447}
Accumulated order: {'order_id': '552fc2b3-ea39-4698-a211-bd98d99294e9', 'customer_id': 'cus-c5661968-261e-4008-a5f8-330d626504be', 'total_weight': 47.33167194989712, 'total_volume': 148.42671794201303, 'total_price': 913.9345274665333, 'order_timestamp': '2024-10-03 03:31:47', 'status': 'RECEIVED', 'lat': 40.51058125835963, 'lon': -3.7204986053130877}
Threshold met: Dispatching 2 orders.


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGSEGV (0xb) at pc=0x0000000107aa51b8, pid=10178, tid=145703
#
# JRE version: OpenJDK Runtime Environment Homebrew (11.0.24) (build 11.0.24+0)
# Java VM: OpenJDK 64-Bit Server VM Homebrew (11.0.24+0, mixed mode, tiered, compressed oops, g1 gc, bsd-aarch64)
# Problematic frame:
# V  [libjvm.dylib+0x6951b8]  ObjectSynchronizer::inflate(Thread*, oopDesc*, ObjectSynchronizer::InflateCause)+0x18c
#
# No core dump will be written. Core dumps have been disabled. To enable core dumping, try "ulimit -c unlimited" before starting Java again
#
# An error report file with more information is saved as:
# /Users/borja/Documents/Somniumrema/projects/de/route_optimizer/notebooks/hs_err_pid10178.log
#
# If you would like to submit a bug report, please visit:
#   https://github.com/Homebrew/homebrew-core/issues
#


Py4JError: An error occurred while calling o986.save

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
